https://qiita.com/akita0724/items/694cc4e142d8d1d7fdaa

| 試験区分                         | コード |
|-----------------------------------|------|
| 応用情報技術者試験                 | ap    |
| ITストラテジスト試験               | st    |
| システムアーキテクト試験             | sa    |
| プロジェクトマネージャ試験           | pw    |
| ネットワークスペシャリスト試験       | nw    |
| データベーススペシャリスト試験         | db    |
| エンベデッドシステムスペシャリスト試験 | es    |
| ITサービスマネージャ試験           | sm    |
| システム監査技術者試験             | au    |
| 情報処理安全確保支援士試験           | sc    |

In [3]:
import requests
import os
import shutil
import tempfile
from os import makedirs, path
from time import sleep
from urllib.request import urlretrieve

# 試験区分とコードの対応表
kamoku_codes = {
    "ap": "応用情報技術者試験",
    "st": "ITストラテジスト試験",
    "sa": "システムアーキテクト試験",
    "pw": "プロジェクトマネージャ試験",
    "nw": "ネットワークスペシャリスト試験",
    "db": "データベーススペシャリスト試験",
    "es": "エンベデッドシステムスペシャリスト試験",
    "sm": "ITサービスマネージャ試験",
    "au": "システム監査技術者試験",
    "sc": "情報処理安全確保支援士試験",
}

# 過去問のURLを生成するための情報
base_url = "https://www.ipa.go.jp/shiken/mondai-kaiotu/"
ids = [
    {"id": "m42obm000000afqx-att", "filename_ids": "2024r06a_"},
    {"id": "m42obm000000afqx-att", "filename_ids": "2024r06h_"},
    {"id": "ps6vr70000010d6y-att", "filename_ids": "2023r05a_"},
    {"id": "ps6vr70000010d6y-att", "filename_ids": "2023r05h_"},
    {"id": "gmcbt80000008smf-att", "filename_ids": "2022r04a_"},
    {"id": "gmcbt80000009sgk-att", "filename_ids": "2022r04h_"},
    {"id": "gmcbt8000000apad-att", "filename_ids": "2021r03a_"},
    {"id": "gmcbt8000000d5ru-att", "filename_ids": "2021r03h_"},
    {"id": "gmcbt8000000d05l-att", "filename_ids": "2020r02o_"},
    {"id": "gmcbt8000000dict-att", "filename_ids": "2019r01a_"},
    {"id": "gmcbt8000000ddiw-att", "filename_ids": "2019h31h_"},
    {"id": "gmcbt8000000f01f-att", "filename_ids": "2018h30a_"},
    {"id": "gmcbt8000000fabr-att", "filename_ids": "2018h30h_"},
    {"id": "gmcbt8000000fqpm-att", "filename_ids": "2017h29a_"},
    {"id": "gmcbt8000000fzx1-att", "filename_ids": "2017h29h_"},
    {"id": "gmcbt8000000g6fw-att", "filename_ids": "2016h28a_"},
    {"id": "gmcbt8000000gn5o-att", "filename_ids": "2016h28h_"},
    {"id": "gmcbt8000000gxj0-att", "filename_ids": "2015h27a_"},
    {"id": "ug65p90000000f52-att", "filename_ids": "2015h27h_"},
    {"id": "ug65p90000000ye5-att", "filename_ids": "2014h26a_"},
    {"id": "ug65p90000001dzu-att", "filename_ids": "2014h26h_"},
    {"id": "ug65p900000027za-att", "filename_ids": "2013h25a_"},
    {"id": "ug65p90000002e6g-att", "filename_ids": "2013h25h_"},
    {"id": "ug65p90000002h5m-att", "filename_ids": "2012h24a_"},
    {"id": "ug65p900000038er-att", "filename_ids": "2012h24h_"},
    {"id": "ug65p90000003ojp-att", "filename_ids": "2011h23a_"},
    {"id": "ug65p90000003ya2-att", "filename_ids": "2011h23tokubetsu_"},
    {"id": "ug65p90000004d6f-att", "filename_ids": "2010h22a_"},
    {"id": "ug65p90000004n2z-att", "filename_ids": "2010h22h_"},
    {"id": "gmcbt8000000f3yi-att", "filename_ids": "2009h21a_"},
    {"id": "ug65p90000009bhl-att", "filename_ids": "2009h21h_"},
]

# ファイルの種類とファイル名のパターン
file_types = {
    "am": ["am_qs", "am_ans"],
    "pm": ["pm_qs", "pm_ans", "pm_cmnt"],
    "am1": ["am1_qs", "am1_ans"],
    "am2": ["am2_qs", "am2_ans"],
    "pm1": ["pm1_qs", "pm1_ans", "pm1_cmnt"],
    "pm2": ["pm2_qs", "pm2_ans", "pm2_cmnt"],
}

# ファイル名のパターン
filename_codes = {
    "qs": "問題",
    "ans": "解答",
    "cmnt": "採点講評",
}


def dl_mondai(kamoku_code):
    kamoku_name = kamoku_codes[kamoku_code]

    for data in ids:
        year = data["filename_ids"][:4]
        season = data["filename_ids"][7]
        season_name = {"h": "春季", "a": "秋季", "o": "特別"}.get(season, "特別")

        # 一時フォルダを作成
        with tempfile.TemporaryDirectory() as temp_dir:
            # 午後の問題ファイルが存在するか確認するためのフラグ
            pm_exists = False
            create_dir = None

            for time_part in ["pm", "am"]: # 午後を先に処理するように変更
                # 午前I/II、午後I/IIのファイルがあるか確認
                for i in range(1, 3):
                    new_time_part = f"{time_part}{i}"

                    # 午前問題の場合、"ap" または "koudo" を使用
                    am_code = "koudo" if kamoku_code != "ap" and time_part == "am" and i == 1 else kamoku_code

                    # URLとファイル名を生成 (問題)
                    new_url_qs = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{new_time_part}_qs.pdf"
                    new_filename_qs = f"{year}{season_name}-{new_time_part.upper()}_{filename_codes['qs']}.pdf"

                    # URLとファイル名を生成 (解答)
                    new_url_ans = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{new_time_part}_ans.pdf"
                    new_filename_ans = f"{year}{season_name}-{new_time_part.upper()}_{filename_codes['ans']}.pdf"

                    # 午後の場合、問題ファイルが存在するか確認
                    if time_part == "pm":
                        temp_file_path_qs = path.join(temp_dir, new_filename_qs)
                        if i == 1: # PM1 の場合のみ存在確認
                            if dl_file(new_url_qs, temp_file_path_qs):
                                pm_exists = True # 午後の問題が存在する場合、フラグを立てる
                                if not create_dir: # create_dir がまだ定義されていない場合
                                    create_dir = f"./DL/{kamoku_name}/{year}/{season_name}"
                                    if not path.exists(create_dir):
                                        makedirs(create_dir)
                                # 解答をダウンロード
                                temp_file_path_ans = path.join(temp_dir, new_filename_ans)
                                dl_file(new_url_ans, temp_file_path_ans)

                                # 採点講評のURLとファイル名を生成 (午後I/IIのみ)
                                if time_part == "pm":
                                    new_url_cmnt = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{new_time_part}_cmnt.pdf"
                                    new_filename_cmnt = f"{year}{season_name}-{new_time_part.upper()}_{filename_codes['cmnt']}.pdf"
                                    temp_file_path_cmnt = path.join(temp_dir, new_filename_cmnt)
                                    dl_file(new_url_cmnt, temp_file_path_cmnt)
                            else:
                                # PM1 が存在しない場合、PM のみ存在するか確認
                                new_time_part_pm = "pm"
                                new_url_qs_pm = f"{base_url}{data['id']}/{data['filename_ids']}{kamoku_code}_{new_time_part_pm}_qs.pdf"
                                new_filename_qs_pm = f"{year}{season_name}-{new_time_part_pm.upper()}_{filename_codes['qs']}.pdf"
                                temp_file_path_qs_pm = path.join(temp_dir, new_filename_qs_pm)
                                if dl_file(new_url_qs_pm, temp_file_path_qs_pm):
                                    pm_exists = True
                                    if not create_dir:
                                        create_dir = f"./DL/{kamoku_name}/{year}/{season_name}"
                                        if not path.exists(create_dir):
                                            makedirs(create_dir)
                                    new_url_ans_pm = f"{base_url}{data['id']}/{data['filename_ids']}{kamoku_code}_{new_time_part_pm}_ans.pdf"
                                    new_filename_ans_pm = f"{year}{season_name}-{new_time_part_pm.upper()}_{filename_codes['ans']}.pdf"
                                    temp_file_path_ans_pm = path.join(temp_dir, new_filename_ans_pm)
                                    dl_file(new_url_ans_pm, temp_file_path_ans_pm)
                                    new_url_cmnt_pm = f"{base_url}{data['id']}/{data['filename_ids']}{kamoku_code}_{new_time_part_pm}_cmnt.pdf"
                                    new_filename_cmnt_pm = f"{year}{season_name}-{new_time_part_pm.upper()}_{filename_codes['cmnt']}.pdf"
                                    temp_file_path_cmnt_pm = path.join(temp_dir, new_filename_cmnt_pm)
                                    dl_file(new_url_cmnt_pm, temp_file_path_cmnt_pm)
                                else:
                                    print(f"{year}{season_name}の午後問題が存在しないため、午前のダウンロードをスキップします。")
                                    break # 午後が存在しない場合、午前のダウンロードをスキップ

                        elif pm_exists: # PM2 の場合、PM1 が存在する場合のみダウンロード
                            # PM2 の問題、解答、採点講評をダウンロード
                            temp_file_path_qs = path.join(temp_dir, new_filename_qs)
                            dl_file(new_url_qs, temp_file_path_qs)
                            temp_file_path_ans = path.join(temp_dir, new_filename_ans)
                            dl_file(new_url_ans, temp_file_path_ans)
                            if time_part == "pm":
                                new_url_cmnt = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{new_time_part}_cmnt.pdf"
                                new_filename_cmnt = f"{year}{season_name}-{new_time_part.upper()}_{filename_codes['cmnt']}.pdf"
                                temp_file_path_cmnt = path.join(temp_dir, new_filename_cmnt)
                                dl_file(new_url_cmnt, temp_file_path_cmnt)
                    # 午前の場合、ダウンロードを実行
                    elif time_part == "am" and pm_exists: # 午後が存在する場合のみ、午前をダウンロード
                        temp_file_path_qs = path.join(temp_dir, new_filename_qs)
                        if dl_file(new_url_qs, temp_file_path_qs):
                            # 解答をダウンロード
                            temp_file_path_ans = path.join(temp_dir, new_filename_ans)
                            dl_file(new_url_ans, temp_file_path_ans)

                # am/pm のファイルがないか確認
                if pm_exists: # 午後が存在する場合のみ、am/pmのファイルを確認
                    for file_type_suffix in file_types[time_part]:
                        # 午前問題の場合、"ap" または "koudo" を使用
                        am_code = "koudo" if kamoku_code != "ap" and time_part == "am" else kamoku_code

                        # 過去問のURLを生成
                        url = f"{base_url}{data['id']}/{data['filename_ids']}{am_code}_{file_type_suffix}.pdf"

                        # ファイル名のパターンを決定
                        if file_type_suffix.endswith("_cmnt"):
                            filename_pattern_key = "cmnt"  # 採点講評の場合
                        else:
                            filename_pattern_key = file_type_suffix.split("_")[1]

                        # 保存するファイル名を生成
                        filename = f"{year}{season_name}-{time_part.upper()}_{filename_codes[filename_pattern_key]}.pdf"
                        temp_file_path = path.join(temp_dir, filename)
                        dl_file(url, temp_file_path)

            if pm_exists:
                # 一時フォルダから正式なフォルダへファイルを移動
                for filename in os.listdir(temp_dir):
                    temp_file_path = path.join(temp_dir, filename)
                    if path.isfile(temp_file_path):
                        shutil.move(temp_file_path, path.join(create_dir, filename))

            if not pm_exists:
                print(f"{year}{season_name}の試験は存在しないため、フォルダを作成しません。")
                continue # 午後が存在しない場合、フォルダを作成しない

    print(f"{kamoku_name}のダウンロードが完了しました。")


# ファイルをダウンロードする関数
def dl_file(url, filename):
    if path.exists(filename):
        print(f"{filename} は既に存在するため、ダウンロードをスキップします。")
        return True

    response = requests.head(url)
    if response.status_code == 200:
        urlretrieve(url, filename)
        print(f"{filename} を保存しました。")
        sleep(3)
        return True
    else:
        print(f"{filename} は見つかりませんでした。")
        return False


# プログラムのエントリポイント
if __name__ == "__main__":
    while True:
        inp = input(
            "ダウンロードしたい試験区分のコードを小文字で入力してください（例：ap）：\n修了する場合はqを入力してください。"
        )
        if inp in kamoku_codes:
            dl_mondai(inp)
            break
        elif inp == "q":
            print("終了します。")
            exit()
        else:
            print("試験区分のコードを入力してください。")

C:\Users\n3oti\AppData\Local\Temp\tmpxzf71td2\2024秋季-PM1_問題.pdf は見つかりませんでした。
C:\Users\n3oti\AppData\Local\Temp\tmpxzf71td2\2024秋季-PM_問題.pdf を保存しました。
C:\Users\n3oti\AppData\Local\Temp\tmpxzf71td2\2024秋季-PM_解答.pdf を保存しました。
C:\Users\n3oti\AppData\Local\Temp\tmpxzf71td2\2024秋季-PM_採点講評.pdf は見つかりませんでした。
C:\Users\n3oti\AppData\Local\Temp\tmpxzf71td2\2024秋季-PM2_問題.pdf は見つかりませんでした。
C:\Users\n3oti\AppData\Local\Temp\tmpxzf71td2\2024秋季-PM2_解答.pdf は見つかりませんでした。
C:\Users\n3oti\AppData\Local\Temp\tmpxzf71td2\2024秋季-PM2_採点講評.pdf は見つかりませんでした。
C:\Users\n3oti\AppData\Local\Temp\tmpxzf71td2\2024秋季-PM_問題.pdf は既に存在するため、ダウンロードをスキップします。
C:\Users\n3oti\AppData\Local\Temp\tmpxzf71td2\2024秋季-PM_解答.pdf は既に存在するため、ダウンロードをスキップします。
C:\Users\n3oti\AppData\Local\Temp\tmpxzf71td2\2024秋季-PM_採点講評.pdf は見つかりませんでした。
C:\Users\n3oti\AppData\Local\Temp\tmpxzf71td2\2024秋季-AM1_問題.pdf を保存しました。
C:\Users\n3oti\AppData\Local\Temp\tmpxzf71td2\2024秋季-AM1_解答.pdf を保存しました。
C:\Users\n3oti\AppData\Local\Temp\tmpxzf71td2\2024秋季-AM2_問題.pdf を保

読み取り
https://qiita.com/kccs_hiroshi-tatsuwaki/items/f3277a157acf965c33e8

In [ ]:
!pip install yomitoku

In [4]:
import os
from pdf2image import convert_from_path

# PDFを画像に変換する関数
def convert_pdf_to_image(pdf_path, output_dir_path):
    os.makedirs(output_dir_path, exist_ok=True)

    images = convert_from_path(pdf_path)
    for i, image in enumerate(images):
        output_path = f"{output_dir_path}/page_{i + 1}.png"
        image.save(output_path, "PNG")
        print(f"Saved: {output_path}")

# ダウンロードしたPDFを画像に変換する処理
def process_pdfs():
    dl_dir = "./DL"
    png_dir = "./PNG"
    os.makedirs(png_dir, exist_ok=True)

    for root, dirs, files in os.walk(dl_dir):
        for file in files:
            if file.lower().endswith(".pdf"):
                pdf_path = os.path.join(root, file)
                # 出力先のディレクトリを作成
                relative_path = os.path.relpath(root, dl_dir)
                output_base_dir = os.path.join(png_dir, relative_path)
                
                # ファイル名から拡張子を除いた部分を取得
                file_name_without_ext = os.path.splitext(file)[0]
                
                # ファイル名と同じ名前のフォルダを最下層に作成
                output_dir = os.path.join(output_base_dir, file_name_without_ext)

                # フォルダが存在するか確認
                if os.path.exists(output_dir):
                    print(f"{output_dir} は既に存在するため、変換をスキップします。")
                    continue
                
                os.makedirs(output_dir, exist_ok=True)
                convert_pdf_to_image(pdf_path, output_dir)
                print(f"{pdf_path} の変換が完了しました。") 

    print("全てのPDFファイルの変換が完了しました。") 

if __name__ == "__main__":
    process_pdfs()

Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_1.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_2.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_3.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_4.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_5.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_6.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_7.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_8.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_9.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_10.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_11.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_12.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_13.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_14.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_15.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_問題/page_16.png
Saved: ./PNG\ITストラテジスト試験\2009\秋季\2009秋季-AM1_解答/page_1.png
Saved: 